In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel("230807_input.xlsx")
df.head()

,year,month,day,weekday,time,q_bld_load,oadb,rh,oawb,mode1,...,temp_chws2,temp_out_br_ch2,mode3,q_load3,temp_chws3,temp_out_br_ch3,mode4,q_load4,temp_chws4,temp_out_br_ch4
0,NaN,NaN,NaN,NaN,23:00:00,NaN,28.5,84.0,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,23:15:00,NaN,28.3,84.2,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,23:30:00,NaN,28.2,84.4,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,23:45:00,NaN,28.1,84.6,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1900-01-01 00:00:00,NaN,28.0,84.8,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN


In [9]:
df['mode1'][0]
lst.append(t_oawb) # [3, 3, 2 ... => 96개
df['new_lst'] = lst

-1.0

In [10]:
import math

In [ ]:
# 열교환기
def hx(train, flow_br):
    flow_chw = [0, 14018, 14018, 14018, 14018]
    ka_hx_ref = [0, 2700, 2250, 2400, 2400]
    kw_chw_pump = [0, 122.2, 129.2, 125.7, 125.7]
    
    q_hx = q_load
    temp_chwr[train] = temp_chws + (q_hx * 3024) / (flow_chw * 60)
    ka_hx[train] = ka_hx_ref[train] * (flow_br/21000) ** (1/3)
    lmtd_hx = (q_hx * 3024) / (ka_hx * 860)
    
    dtemp_br_hx = (q_hx * 3024) / (1.038 x 0.8975 x flow_br * 60)
    temp_in_br_hx = (temp_chws - (t_chwr - dtemp_br_hx) x math.exp((temp_chws - temp_chwr + dtemp_br_hx) / lmtd)) / (1 - math.exp((temp_chws - t_chwr + dtemp_br_hx) / lmtd))
    temp_out_br_hx = temp_in_br_hx + dtemp_br_hx
    
    return temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump

In [11]:
# 냉동기, 냉각탑
def chiller(train, temp_in_br_ch, temp_out_br_ch, flow_br):
    flow_cw = [0, 18900, 19000, 18800, 19000]
    q_ch_rated = [0, 1118.2, 1118.2, 1118.2, 1118.2]
    kw_ch_rated = [0, 952.6, 952.6, 952.6, 952.6]
    kw_cw_pump = [0, 156.9, 161.4, 154.7, 154.7]
    kw_ct_fan = [0, 58.8, 47.8, 61.6, 61.6]
    
    a = [0, 1.458645293, 0.040009978, -1.36368*(10**(-6)), -0.02013792, 2.06332*(10**(-6)), 2.26211*(10**(-7))]
    b = [0, 1.062254257, 0.037469286, 0.001538748, -0.040480028, 0.001323365, -0.002722094]
    c = [0, 0.4, 0.3, 0.3]
    d = [0, 3.51525454, -0.083236649, 0.000265981, 3.966633299, -0.152643525, 0.002041101, -0.217416145, 0.013288222, -0.000281392]
    
    for temp_cwr in range(oawb+10, oawb+3, -0.02):
        cap_ft = a[1] + a[2] * temp_out_br_ch + a[3] * temp_out_br_ch**2 + a[4] * temp_cwr + a[5] * temp_cwr**2 + a[6] * temp_out_br_ch * temp_cwr
        q_ch_available = cap_ft * q_ch_rated[train] * 3024
    
        # 냉방 운전시, 냉동기 입구 브라인 온도(T_in_BR_CH#)는 열교환기 계산에서 구한 열교환기 출구 브라인온도(T_out_BR_HX#)와 같다. 
        
        q_ch_current = flow_br * 60 * 1.038 * 0.8975 * (temp_in_br_ch - temp_out_br_ch)
        plr = q_ch_current / q_ch_available
        
        if plr > 1:
            plr = 1
            q_ch_current = q_ch_available
            temp_out_br_ch = temp_in_br_ch - q_ch_current / (flow_br * 60 * 1.038 * 0.8975)
            # continue
        
        # 냉방 운전시, T_out_BR_CH#(=T_in_BR_IceStor#)가 맞지 않으면 3항의 축열조 계산을 다시 실행
        # 축열조 계산을 다시하면 브라인 유량이 달라지므로 냉동기 계산도 다시 실행
        
        eir_ft = b[1] + b[2] * (temp_out_br_ch) + b[3] * temp_out_br_ch**2 + b[4] * temp_cwr + b[5] * temp_cwr**2 + b[6] * temp_out_br_ch * temp_cwr
        kw_ch_available = kw_ch_rated[train] * eir_ft * cap_ft
        eir_fplr = c[1] + c[2] * plr + c[3] * plr**2
        kw_ch_current = kw_ch_available * eir_fplr
        q_cw_current = q_ch_current + kw_ch_current * 860
        
        temp_range = q_cw_current / (flow_cw * 60)
        temp_approach = d[1] + d[2]*oawb + d[3]*oawb**2 + d[4]*temp_range + d[5]*temp_range*oawb + d[6]*oawb**2*temp_range + d[7]*temp_range**2 + d[8]*oawb*temp_range**2 + d[9]*oawb**2*temp_range**2
        temp_cws = oawb + temp_approach
        
        temp_cwr_calc = oawb + temp_approach + temp_range
        
        if abs(temp_cwr_calc - temp_cwr) < 0.01:
            break
    temp_cwr = temp_cwr_calc
    
    return plr, temp_out_br_ch, temp_in_br_ch, q_ch_current, q_ch_available, kw_ch_current, q_cw_current, temp_cws, temp_cwr
        

In [2]:
def br_flow(open_icestor_mixvalve):
    # [0, 0, 0, 0, 0]
    flow_br = [0] * 5 
    flow_br_open0 = [0, 21858, 21000, 20373, 21000]
    flow_br_open100 = [0, 12166, 13206, 7500, 13000]
    flow_br[train] = flow_br_open0[train] - ((flow_br_open0[train] - flow_br_open100[train] * (open_icestor_mixvalve/100)))
    return flow_br
    
def kw_br_pump(open_icestor_mixvalve):
    kw_br_pump = [0] * 5
    kw_br_pump_open0 = [0, 213.5, 199.5, 210.9, 210.9]
    kw_br_pump_open100 = [0, 195.0, 186.8, 175.3, 175.3]
    kw_br_pump[train] = kw_br_pump_open0[train] - ((kw_br_pump_open0[train] - kw_br_pump_open100[train]) * (open_icestor_mixvalve / 100)))
    return kw_br_pump

for time in range(1, 97):
    # 기온(T_OADB)과 습도(%RH)를 읽어 와서 습공기 계산공식으로부터 외기습구온도(T_OAWB)를 구한다
    # 전체 냉방부하를 각 Train#로 나누는 방법은 ?
    
    for train in range(1, 5):
        isStart = True
        if mode == -1: # 제빙 운전
            open_icestor_mixvalve = 100
            flow_br = br_flow(open_icestor_mixvalve)
            kw_br_pump = kw_br_pump(open_icestor_mixvalve)
            
            for t_min in range(1, 16):
                if isStart == True:
                    temp_in_br_ch = temp_water_icestor
                    isStart = False
                else:
                    temp_in_br_ch = temp_br_mix_icestor
                temp_out_br_ch = df['temp_out_br_ch'+str(train)][time]
                plr, temp_out_br_ch, temp_in_br_ch, q_ch_current, q_ch_available, kw_ch_current, q_cw_current, temp_cws, temp_cwr = 
                chiller(train, temp_in_br_ch, temp_out_br_ch, flow_br[train])
                # "축열조 제빙계산"으로 가서 계산
                
            # 계산이 모두 끝나면, 계산과정에서 구한 모든 변수값을 계산한 timestep과 함께 저장
        elif mode == 2: # 병렬 운전
            kwh_br_pump = [0] * 5
            kwh_chw_pump = [0] * 5
            for t_min in range(1, 16):
                open_icestor_mixvalve = 100
                flow_br = br_flow(open_icestor_mixvalve)
                kw_br_pump = kw_br_pump(open_icestor_mixvalve)
                kwh_br_pump[train] += kw_br_pump / 60
                
                temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump = hx(train, flow_br)
                kwh_chw_pump[train] += kw_chw_pump / 60
                # 축열조 방열 계산
                # 냉동기와 냉각탑 계산
        elif mode == 1.2: # 냉동기 단독 운전
            kwh_br_pump = [0] * 5
            kwh_chw_pump = [0] * 5
            for t_min in range(1, 16):
                open_icestor_mixvalve = 0
                flow_br = br_flow(open_icestor_mixvalve)
                kw_br_pump = kw_br_pump(open_icestor_mixvalve)
                
                temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump = hx(train, flow_br)
                kwh_chw_pump[train] += kw_chw_pump / 60
                # 냉동기와 냉각탑 계산
        elif mode == 0.8: # 축열조 단독 운전
            kwh_br_pump = [0] * 5
            kwh_chw_pump = [0] * 5
            for t_min in range(1, 16):
                open_icestor_mixvalve = 100
                flow_br = br_flow(open_icestor_mixvalve)
                kw_br_pump = kw_br_pump(open_icestor_mixvalve)
                
                temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump = hx(train, flow_br)
                kwh_chw_pump[train] += kw_chw_pump / 60
                # 축열조 방열 계산
        elif mode == 0: # 운전 정지
            # 이전 값 참조
        else:
            pass

SyntaxError: unmatched ')' (2754198852.py, line 12)